In [ ]:
#FUNCTION

def loadbalancing(time,reqcs):
    import pandas as pd
    distancedf = pd.read_csv('distance.csv')
    masterdf   = pd.read_csv('MasterSheet.csv')
    logdf = pd.read_csv("log.csv")
    cscount  = len(masterdf.index)
    i=1
    flag=0
    while(i<=cscount):
        cs = "CS_"+str(i)
        if(cs!=reqcs):
            df = pd.read_csv("cs_data\\"+cs+".csv")
            df.set_index('Hour', inplace=True)
            df = df[df.index==time]
            if(flag==0):
                flag= 1
                dff = df
            else:
                dff = dff.append(df)
        i=i+1
    station = []
    temp_distance = 0
    for index, rows in masterdf.iterrows(): 
        s_list = rows.Station_ID
        if(s_list!=reqcs):
            station.append(s_list)    
    dff['Station'] = station
    dff['Distance'] = temp_distance
    dff.set_index('Station', inplace=True)
    distance = distancedf[distancedf['L1'] == reqcs]
    for indexa, rowsa in dff.iterrows(): 
        for indexb, rowsb in distance.iterrows():
            if(indexa==rowsb.L2):
                dff.at[indexa,'Distance']=distance.at[indexb,'Distance']
    dff.to_csv('Analysis\T'+str(time)+'.csv')
    dff.sort_values("Amount", axis = 0, ascending = True, inplace = True)
    dff.sort_values("Distance", axis = 0, ascending = True, inplace = True)
    dff.sort_values("Dirty_bit", axis = 0, ascending = True, inplace = True)
    dff = dff.reset_index()
    dff.Dirty_bit[0] = 1
    station = dff.at[0,'Station']
    
    
    reqdf = pd.read_csv("cs_data\\"+reqcs+".csv")
    resdf = pd.read_csv("cs_data\\"+station+".csv")
    reqdf.set_index('Hour', inplace=True)
    resdf.set_index('Hour', inplace=True)
    reqdf.Energy[time] = reqdf.Energy[time] + reqdf.Amount[time]
    resdf.Energy[time] = resdf.Energy[time] - reqdf.Amount[time]
    reqdf.Amount[time] = reqdf.Predicted[time] - reqdf.Energy[time]
    resdf.Amount[time] = resdf.Predicted[time] - resdf.Energy[time]
    resdf.Dirty_bit[time]=1
    datadf = pd.DataFrame(reqdf)
    datadf.to_csv('cs_data\\'+reqcs+'.csv')
    datadf = pd.DataFrame(resdf)
    datadf.to_csv('cs_data\\'+station+'.csv')

    
    log = {'Time':[time],'Req Cs':[reqcs],'From CS':[station]}
    logdf = pd.DataFrame(log)
    logdf.set_index('Time', inplace=True)
    logdf.to_csv('log.csv', mode='a', header=False)   